In [240]:
import string 
import random 
from numpy.random import choice
from scipy.stats import skewnorm
import pandas as pd

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import csv


import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import next_chars
USE_TRUE_PROB = False
USE_NFA = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [64]:
def split(word):
    return [char for char in word]

test= pd.read_csv("test_data.txt", sep='\n', header=None, names = ['regex'])
train = pd.read_csv("train_data.txt", sep='\n', header=None, names = ['regex'])
train_ten = train.loc[train.index.repeat(10)].reset_index(drop=True)
with open('train_data_pairs.txt') as f:
    lines = f.readlines()



examples = [split(line.strip()) for line in lines]

print(len(examples))
train_ten["example"] = examples 

1000000


In [110]:
if USE_TRUE_PROB:
  pairs = [(eval(row["regex"]),eval(row["regex"])) for i, row in train_ten.iterrows()]
else :
  pairs = [(eval(row["regex"]), row["example"]) for index,row in train_ten.iterrows()]


In [111]:
def generate_training_data(num_examples: int, max_size:int, regex_things:list) -> list:
  examples = [ [ regex_things[random.randint(0,len(regex_things)-1)] for  i in range(random.randint(1,max_size))] for k in range(num_examples)]
  return examples
  
def generate_unambig_training_data(num_examples: int, max_size:int, regex_things:list) -> list:
  examples = [ ]
  for  i in range(num_examples):
    count = 0
    digit_plus = False 
    low_letter_plus = False
    high_letter_plus = False
    alphanum_plus = False
    max_regex_size = random.randint(1,max_size)
    regex = []
    while count < max_regex_size:
      num = random.randint(0,2)
      if num < 1:
        atom = regex_only[random.randint(0,len(regex_only)-1)]
      else:
        atom = regex_things[random.randint(0,len(regex_things)-1)]
      if digit_plus:
        if re.search(atom, '9') and  re.search(atom, '1'):
          continue
      if low_letter_plus:
        if re.search(atom, 'a') and re.search(atom, 'b'):
          continue
      if high_letter_plus:
        if re.search(atom, 'A') and re.search(atom, 'B') :
          continue
      if high_letter_plus:
        atom = regex_things[10:][random.randint(0,len(regex_things[10:])-1)]
      regex.append(atom)
      digit_plus, low_letter_plus, high_letter_plus,  alphanum_plus  = False , False , False, False
      if atom[-1] == '+':
        if atom == '[0-9]+':
          digit_plus = True 
        elif atom == '[a-z]+':
          low_letter_plus = True
        elif atom == '[A-Z]+':
          high_letter_plus = True
        else:
          alphanum_plus = True
      count+=1
      continue 
    examples.append(regex)
  return examples
 
def generate_training_data_pairs(training_data: list) -> list:
  #for i in training_data:
    #print(i)
    #hypothesis.strategies.from_regex('^' + ''.join(i) + '$').example().strip()
  return [(i,i) for i in training_data]
  #return [(i, hypothesis.strategies.from_regex('^' + ''.join(i) + '$').example().strip()) for i in training_data]

In [112]:
random.randint(0,2)

1

In [113]:
# make the language 
# first iteration = very restricted language no optionals no constants
regex_only : list = ['[0-9]','[a-z]','[A-Z]','[a-zA-Z]', '[a-zA-Z0-9]', '[0-9]+','[a-z]+','[A-Z]+','[a-zA-Z]+', '[a-zA-Z0-9]+']
ascii_char : list = list(string.printable)[:95]
special_char = ['!','"','#','%','&',"'",',','-','.',':',';',
 '<','>','@','_','`',' ']
#regex_things = regex_things + [ i for i in ascii_char if i not in special_char]
regex_things = regex_only + ascii_char[:65] + special_char
# takes some time to run 
# TODO: Write this into a file and read from a file
#training_data = generate_unambig_training_data(100000, 5, regex_things)
#pairs = generate_training_data_pairs(training_data)

In [114]:
SOS_token = 0
EOS_token = 1

# constructing langiage using 
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
      
    def addList(self, list_words):
        for word in list_words:
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [196]:
MAX_LENGTH = 25
def readLangs(lang1: list, lang2:list, reverse=False):
  input_lang = Lang('regex')
  output_lang = Lang('english')
  input_lang.addList(lang1)
  output_lang.addList(lang2)
  print(input_lang.name, input_lang.n_words)
  print(output_lang.name, output_lang.n_words)
  return input_lang, output_lang


In [197]:
input_lang, output_lang = readLangs(regex_things, ascii_char)

regex 91
english 97


In [198]:
# model code also taken from 
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [199]:
def indexesFromList(lang, char_list):
    return [lang.word2index[word] for word in char_list]


def tensorFromList(lang, char_list):
    indexes = indexesFromList(lang, char_list)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def pairFromPair(pair):
    input_tensor = tensorFromList(input_lang, pair[0])
    #target_tensor = tensorFromList(output_lang, list(pair[1]))
    return (pair[0], input_tensor)

def tripleFromPair(pair):
    input_tensor = tensorFromList(input_lang, pair[0])
    target_tensor = tensorFromList(output_lang, pair[1])
    return (pair[0], input_tensor, target_tensor)


In [303]:
import next_chars
USE_TRUE_PROB = True
USE_NFA = True

In [330]:
teacher_forcing_ratio = 0.5

#changed this to have an input_regex
def train(input_regex, input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    #target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    #use_true_probabilities = False
    #if random.random() < teacher_forcing_ratio else False

    # Teacher forcing: Feed the target as the next input
    if USE_TRUE_PROB:
      if USE_NFA:
        state = next_chars.regex_to_nfa(input_regex)
        chosen_token = np.inf
        count = 0
        total = 0
        reached_end = False
        while chosen_token!=EOS_token and count < MAX_LENGTH:
        #print(count)
            count +=1
            decoder_output, decoder_hidden, decoder_attention = decoder(
              decoder_input, decoder_hidden, encoder_outputs)
            poss_chars = next_chars.possible_next_chars(state)

            indexed_chars = indexesFromList(output_lang, poss_chars)
            true_tensor = torch.zeros(output_lang.n_words)
            true_tensor = true_tensor.to(device)
            if next_chars.end_token_is_allowed_here(state):
              indexed_chars.append(EOS_token)
            for i in indexed_chars:
              true_tensor[i] = 1/ len(indexed_chars) 
        # create a tensor that gives equal probabilities to next chars 
            loss += criterion(decoder_output, true_tensor.reshape(1, output_lang.n_words))
            chosen_token = random.choice(indexed_chars)
            next_tensor = torch.zeros(output_lang.n_words)
            next_tensor[chosen_token] = 1
            if (chosen_token == EOS_token):
                reached_end = True 
                break
            state = next_chars.consume_a_char(state, output_lang.index2word[chosen_token])
        #print(output_lang.index2word[chosen_token])
        #re.complie(#amount of regex we have consumed so far)
        # 0 or 1
        #chosen_tensor = torch.tensor(chosen_token)
            decoder_input = torch.tensor(chosen_token, device = device)  # Teacher forcing
      else:
        nfa = next_chars.regex_to_nfa(input_regex)
        dfa = next_chars.DFA(nfa)
        state = dfa.nodes[0]
        ch = np.inf
        count = 0
        reached_end = False
        while ch!='END' and count < MAX_LENGTH:
        #print(count)
            count +=1
            decoder_output, decoder_hidden, decoder_attention = decoder(
              decoder_input, decoder_hidden, encoder_outputs)
            poss_chars = list(state.transitions.keys())

            indexed_chars = indexesFromList(output_lang, poss_chars)
            true_tensor = torch.zeros(output_lang.n_words)
            true_tensor = true_tensor.to(device)
            #if next_chars.end_token_is_allowed_here(state):
              #indexed_chars.append(EOS_token)
            for index, i in enumerate(indexed_chars):
              true_tensor[i] = state.transitions[poss_chars[index]][1]
            true_tensor[EOS_token] = state.p_end
            loss += criterion(decoder_output, true_tensor.reshape(1, output_lang.n_words))
            ch = np.random.choice(np.array(list(state.transitions.keys()) + ['END']),
                    p=np.array(list(w for j, w in state.transitions.values()) +
                        [state.p_end]))
            if ch == 'END':
                reached_end = True 
                break
            # move the dfa
            state = dfa.nodes[state.transitions[ch][0]]
            # get index of chose character 
            chosen_index = output_lang.word2index[ch]
            next_tensor = torch.zeros(output_lang.n_words)
            next_tensor[chosen_index] = 1

    else:
       reached_end = True 
       count =0
       target_length = target_tensor.size(0)
       #print(target_length)
       for di in range(target_length):
            count+=1
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            #print(decoder_output)
            #print(target_tensor[di])
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing
            if count >= MAX_LENGTH:
              reached_end = False 
        


    if reached_end:
       loss.backward()
     
       encoder_optimizer.step()
       decoder_optimizer.step()




    return loss.item() / count

In [331]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [332]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    if USE_TRUE_PROB:
      training_tuple = [pairFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    else:
      training_tuple = [tripleFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.CrossEntropyLoss()

    for iter in range(1, n_iters + 1):
        #print(training_tuple[iter - 1])
        ex_tuple = training_tuple[iter - 1]
        input_regex = ex_tuple[0]

        input_tensor = ex_tuple[1]
        if not USE_TRUE_PROB:
          output_tensor = ex_tuple[2]
        else: 
          output_tensor = None

        loss = train(input_regex, input_tensor, output_tensor,  encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [333]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [334]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromList(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [335]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

h =trainIters(encoder1, attn_decoder1, 35000, print_every=1000)

0m 20s (- 11m 47s) (1000 2%) 3.9845
0m 40s (- 11m 0s) (2000 5%) 3.7405
0m 59s (- 10m 33s) (3000 8%) 3.5304
1m 18s (- 10m 9s) (4000 11%) 3.3026
1m 38s (- 9m 49s) (5000 14%) 2.9997
1m 58s (- 9m 31s) (6000 17%) 2.7873
2m 18s (- 9m 13s) (7000 20%) 2.6538
2m 36s (- 8m 49s) (8000 22%) 2.4228
2m 56s (- 8m 29s) (9000 25%) 2.4237
3m 15s (- 8m 9s) (10000 28%) 2.3635
3m 35s (- 7m 49s) (11000 31%) 2.3816
3m 54s (- 7m 29s) (12000 34%) 2.2407
4m 14s (- 7m 10s) (13000 37%) 2.2509
4m 33s (- 6m 50s) (14000 40%) 2.1684
4m 53s (- 6m 31s) (15000 42%) 2.2938
5m 13s (- 6m 12s) (16000 45%) 2.1814
5m 32s (- 5m 52s) (17000 48%) 2.1763
5m 52s (- 5m 32s) (18000 51%) 2.1928
6m 11s (- 5m 12s) (19000 54%) 2.1982
6m 30s (- 4m 53s) (20000 57%) 2.1707
6m 49s (- 4m 33s) (21000 60%) 2.1569
7m 10s (- 4m 14s) (22000 62%) 2.2100
7m 29s (- 3m 54s) (23000 65%) 2.1523
7m 48s (- 3m 34s) (24000 68%) 2.1430
8m 7s (- 3m 15s) (25000 71%) 2.0715
8m 27s (- 2m 55s) (26000 74%) 2.1155
8m 46s (- 2m 35s) (27000 77%) 2.1185
9m 4s (- 2m 1

In [342]:
#testing_data = generate_unambig_training_data(1000, 5, regex_things)
count = 0
end = 0
end_correct=0
for index, row in test.iterrows():
  regex_list = eval(row["regex"])
  answer = evaluate(encoder1, attn_decoder1, regex_list)
  if re.search('^' + ''.join(regex_list) + '$', ''.join(answer[0][:-1])):
      count+=1
      if answer[0][-1] == "<EOS>":
         end_correct+=1
      print(f'CORRECT, regex: {"".join(regex_list)}, generated: {"".join(answer[0][:-1])}')
  else:
    print(f'WRONG, regex: {"".join(regex_list)}, generated: {"".join(answer[0][:-1])}')
  if answer[0][-1] == "<EOS>":
    end+=1
print((count/1000) * 100)
print(end/1000)

WRONG, regex: 5[A-Z][0-9][a-zA-Z0-9], generated: 55Y5M
WRONG, regex: G[A-Z]+-, generated: GGGGGTGLGLTGLLLDLLTGLLTG
CORRECT, regex: t[a-zA-Z], generated: tt
WRONG, regex: [A-Z]+ , generated:  
WRONG, regex: [A-Z]+@[a-zA-Z][a-z]l, generated: DHEHCLHLLEHLECLHElllllll
WRONG, regex: 9#!f, generated: 9!#f
CORRECT, regex: [0-9], generated: 4
CORRECT, regex: [0-9][a-z]+, generated: 1vqahnhbhnhdonxvlllnxvll
WRONG, regex: j[a-zA-Z]X_[a-zA-Z0-9]+, generated: jXX-VXMMLMLMEMGGEGMTMTME
CORRECT, regex: [A-Z], generated: I
CORRECT, regex: b, generated: b
WRONG, regex: [a-zA-Z]Z`[a-z]+, generated: Z`Zaxllllllllllllxllllll
CORRECT, regex: &, generated: &
CORRECT, regex: T, generated: T
WRONG, regex: [a-z]+GYgb, generated: bbgbgbgbgbgbbgbgbgbxgbgb
CORRECT, regex: [a-z]N#'b, generated: bN#'b
WRONG, regex: Lk[A-Z], generated: LLk
CORRECT, regex: !kr[a-zA-Z], generated: !krk
CORRECT, regex: [A-Z]`P, generated: P`P
WRONG, regex: [0-9][a-z]+"@0, generated: 0"zxsxsqxsaxsxsxhxhhxhah
WRONG, regex: [a-zA-Z]+I[0-9

In [337]:
testing_data = generate_unambig_training_data(1000, 5, regex_things)

In [338]:
import csv 

with open('test_data.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(testing_data)

In [339]:
MAX_LENGTH

25

In [340]:
answer[0]

[':', "'", '#', 'J', ';', '<EOS>']

In [341]:

'$', ''.join(answer)

TypeError: ignored

In [ ]:
with open('train_data.txt', 'w') as f:
    for item in training_data:
        # write each item on a new line
        f.write("%s\n" % item)

In [ ]:
with open('test_data.txt', 'w') as f:
    for item in testing_data:
        # write each item on a new line
        f.write("%s\n" % item)